### 🔍 Objective:
This project will **introduce you to CNNs**.

### 📌 What You’ll Do:
1. Define suitable transforms/augmentations for your `train` and `test` images.
2. Pass these images into PyTorch `DataLoaders` for batch processing.
3. Implement `CNN` class architecture for pneumonia image classification.
4. Train and validate your model.

💡 **PLEASE PLEASE PLEASE look things up!!! This is YOUR learning experience.**

---

In [3]:
import torch
import kagglehub
import torch.nn as nn

path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

train_path = f'{path}/chest_xray/train/'
test_path = f'{path}/chest_xray/test/'

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.29G/2.29G [00:48<00:00, 50.6MB/s]

Extracting files...


#### 📌 ***TASK 1 - DATA PREPROCESSING***

Define image augmentations in the cell below using two variables:  

- **`transform_train`**: Stores transforms for training images. You can include any augmentations you prefer.  
- **`transform_test`**: Stores transforms for your test images. As a best practice, limit these transformations to only the essential ones from `transform_train`.

Lastly, be sure to convert all images to [tensors](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) via the `transforms.ToTensor()` transform. Don't know transforms? [Click here](https://pytorch.org/vision/stable/transforms.html).

In [7]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# 🔧 Image transforms
transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

transform_test = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# 🔁 Load the dataset
num_batches = 32

train_dataset = ImageFolder(root=train_path, transform=transform_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=num_batches, shuffle=True)

test_dataset = ImageFolder(root=test_path, transform=transform_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=num_batches, shuffle=True)
images, labels = next(iter(train_loader))
print(f"Image batch shape: {images.shape}")
print(f"Label batch shape: {labels.shape}")


Image batch shape: torch.Size([32, 3, 128, 128])
Label batch shape: torch.Size([32])


Now, we'll pass our images and transforms into a `DataLoader`, which allows us to train our model in batches.
Most of the code is done for you, but [click here](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) to learn more.

In [9]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

num_batches = 32

train_dataset = ImageFolder(root=train_path, transform=transform_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=num_batches, shuffle=True)

test_dataset = ImageFolder(root=test_path, transform=transform_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=num_batches, shuffle=True)

#### 📌 ***TASK 2 - CNN Architecture***

This is where you have all the creative freedom in the world. Here are some good questions to ask yourself:

- How many [channels](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-49AG77e4Qp2e7EkARdFsTA) should go into the input layer?
- What measures can I take to avoid [overfitting](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-YdAbhqQzRZaEq39BEQzA6w)?
- What matters to me? (Training Speed / Performance tradeoffs)
- **CONVOLUTION. ACTIVATION FUNCTION. POOLING!!!** 📢📢📢

Not comfortable with PyTorch? [Here](https://youtu.be/mozBidd58VQ?si=TE2_81TEQko1eDXT). Go and make me the best [CNN](https://www.datacamp.com/tutorial/introduction-to-convolutional-neural-networks-cnns) I've ever seen :)

In [10]:
import torch
import torch.nn
image_width = 128
image_height = 128

class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)

        # After 3 poolings on 128x128 → 16x16
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # -> 64x64
        x = self.pool(torch.relu(self.conv2(x)))  # -> 32x32
        x = self.pool(torch.relu(self.conv3(x)))  # -> 16x16
        x = x.view(-1, 64 * 16 * 16)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### 📌 ***TASK 3 - DEFINE TRAIN FUNCTION***  

Define `process_forward_phase` and `train` to update model weights with each new [epoch/iteration](https://www.perplexity.ai/search/i-m-a-curious-naiss-mlb-studen-7SJECNYrS1iYxUR032dp7A). Here are the steps:

- **Forward pass:** Here, our batch is taken through the network to output a prediction (Normal/Pneumonia)
- **Backward pass:** The model goes "What's our loss? Hmmm... Not quite what I want. This means my `weights` aren't adjusted properly. Let me propagate my `loss` backward in hopes of correcting my weights."

We use **`f1_score`** as the primary metric and also display **`accuracy`** for comparison. Most steps are outlined for you—just follow the structure provided!

In [11]:
from tqdm import tqdm # Visualize training progress
from sklearn.metrics import f1_score, accuracy_score
def process_forward_pass(model, batch, criterion):
    images, labels = batch
    labels = labels.float()

    outputs = model(images)
    loss = criterion(outputs, labels.long())
    preds = torch.argmax(outputs, dim=1)

    return loss, preds, labels

def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        all_preds, all_labels = [], []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            optimizer.zero_grad()

            loss, preds, labels = process_forward_pass(model, batch, criterion)
            loss.backward()
            optimizer.step()

            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds)
        print(f"Acc={accuracy:.2f}%, F1={f1:.4f}")



After your model trains, you want to see how well it performs on **unseen data.** Meaning, if this were a live hospital NEEDING your predictions to classify patients with pneumonia, how well would it do?


You simply have to run this cell; all the code is implemented for you (Assuming `process_forward_phase` works fine). 😊

In [ ]:
def test(model, test_loader, criterion):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in test_loader:
            loss, preds, labels = process_forward_phase(model, batch, criterion)
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Final Test Results: Acc={accuracy:.2f}%, F1={f1:.4f}")

#### 📌***TASK 4 - TRAIN MODEL***

We're close!!! We simply need to instantiate the `model`, define a suitable `criterion` (loss), and use an `optimizer` (thing to speed up backpropagation).

In [12]:
import torch.optim as optim

model = PneumoniaCNN()

criterion = nn.CrossEntropyLoss()


optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)


train(model, train_loader, criterion, optimizer, epochs=5)

Epoch 1/5: 100%|██████████| 163/163 [02:24<00:00,  1.12it/s]


Acc=0.86%, F1=0.9132


Epoch 2/5: 100%|██████████| 163/163 [02:24<00:00,  1.12it/s]


Acc=0.94%, F1=0.9622


Epoch 3/5: 100%|██████████| 163/163 [02:26<00:00,  1.11it/s]


Acc=0.95%, F1=0.9676


Epoch 4/5: 100%|██████████| 163/163 [02:32<00:00,  1.07it/s]


Acc=0.96%, F1=0.9714


Epoch 5/5: 100%|██████████| 163/163 [02:36<00:00,  1.04it/s]

Acc=0.96%, F1=0.9702


In [14]:
def test(model, test_loader, criterion):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in test_loader:
            loss, preds, labels = process_forward_pass(model, batch, criterion)
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Final Test Results: Acc={accuracy:.2f}%, F1={f1:.4f}")

Last step: evaluate your model's performance. Remember, you get **1,000,000** brownie points 🍫 if you beat Adam's **`f1_score:`0.8549**.

In [15]:
test(model, test_loader, criterion)

Final Test Results: Acc=0.71%, F1=0.8117
